In [2]:
import PyPDF2, json, pprint, json, os
from llama_index.llms.openai import OpenAI
import os, yaml, warnings, nest_asyncio, random
from llama_index.core import Settings, ChatPromptTemplate
from llama_index.core.llms import (
                                ChatMessage,
                                MessageRole
                                )
warnings.filterwarnings(
                        "ignore", 
                        category=FutureWarning
                        )
nest_asyncio.apply()

In [3]:
"""
- Generate Questions and Answers (Should be adaptive)
- Check the validity / relatedness of the questions to context 
- Check also grammer 
- Feedbacks
"""

'\n- Generate Questions and Answers (Should be adaptive)\n- Check the validity / relatedness of the questions to context \n- Check also grammer \n- Feedbacks\n'

In [4]:
with open('secrets.yaml') as f:
    secrets = yaml.load(f, Loader=yaml.FullLoader)

os.environ["OPENAI_API_KEY"] = secrets['OPENAI_API_KEY']
llm = OpenAI(
            model="gpt-4o", #Model
            api_key=os.environ["OPENAI_API_KEY"],
            temperature=0.3
            )
Settings.llm = llm

In [5]:
QGEN_PROMPT_TEMPLATE = '''
You are an expert in creating educational content, specializing in creating questions / assignments for English language.

Requirements:

- There are 3 question categories.
    1. Multiple Choice Questions (MCQs)
    2. Structured Questions
    3. Fill-in-the-Blanks Questions
- You should generate minimum 2, maximum 5 questions for each category.
- The question should meet the difficulty level as "{difficulty}
- For MCQs 
        -> provide **four** answer options labeled A, B, C, and D. 
        -> Only **one** of the options should be the correct answer.
        -> The incorrect options (distractors) should be plausible to someone unfamiliar with the topic but clearly incorrect to someone who knows the material.
        -> After listing the options, clearly indicate the correct answer by stating "Answer: X" where X is A, B, C, or D.
- For Structured Questions
        -> provide a answer that is a short paragraph.
- For Fill-in-the-Blanks Questions
        -> space should indicate using <blank> tag.
        -> provide a short paragraph with a blank spaces. each question must contain atleast two blanks.
        -> seperately provide the answers for the blanks.
- Ensure that the content is accurate and free of errors.

Example Format:

## Multiple Choice Questions (MCQs)
{
    "Question": <Your question here>,
    "options": ["Option A", "Option B", "Option C", "Option D"],
    "correct_answer": <Your answer here>
}

## Structured Questions
{
    "Question": <Your question here>,
    "Answer": <Your answer here>
}

## Fill-in-the-Blanks Questions
{
    "Question": <Your question here>,
    "Blanks": ["<blank>", "<blank>"],
}

Using above template, generate questions for the following context:

-----------------------------------------
{context}
-----------------------------------------

ALL QUESTIONS MUST RESULT UNDER ONE PYTHON LIST OF DICTIONARIES. DO NOT INCLUDE ANY CODE OUTSIDE THE DICTIONARIES.
'''

qgen_template = ChatPromptTemplate(
                                    message_templates=[
                                                    ChatMessage(
                                                                role=MessageRole.SYSTEM, 
                                                                content=QGEN_PROMPT_TEMPLATE
                                                                )
                                                    ]
                                    )

In [6]:
def read_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf = PyPDF2.PdfReader(file)
        text = ''
        for page in range(len(pdf.pages)):
            text += pdf.pages[page].extract_text()
    return text

def post_process_output(raw_output):
    raw_output = raw_output.strip()
    if (not raw_output.startswith("[")) or (not raw_output.endswith("]")):
        index_start = raw_output.find("[")  
        index_end = raw_output.rfind("]") + 1
        raw_output = raw_output[index_start:index_end]
        json_output = eval(raw_output)
        return json_output
    
    return eval(raw_output)

def generate_quiz(
                file_path,
                difficulty,
                generated_q_dir
                ):
    filename = file_path.split('/')[-1].replace('.pdf', '')
    gen_q_path = f"{generated_q_dir}/{filename}.json"
    if not os.path.exists(gen_q_path):
        while True:
            try:
                text = read_pdf(file_path)
                fmt_messages = qgen_template.format_messages(
                                                            context=text,
                                                            difficulty=difficulty
                                                            )
                chat_response = llm.chat(fmt_messages)
                raw_output = chat_response.message.content
                json_output = post_process_output(raw_output)
                
                with open(gen_q_path, 'w') as f:
                    json.dump(json_output, f, indent=4)

                break

            except Exception as e:
                print("Retrying due to error: ", e)

    with open(gen_q_path) as f:
        questions = json.load(f)
    return questions

In [21]:
raw_output = generate_quiz(
                        file_path = "English/bbc_exam_skills_revising_part1_2.pdf",
                        difficulty = "Medium", generated_q_dir="Generated/Medium/"
                        )
pprint.pprint(raw_output)

[{'Question': 'What is one of the key tips mentioned for effective revision?',
  'correct_answer': 'Rewrite notes and use them for revision',
  'options': ['Leave revision until the last minute',
              'Rewrite notes and use them for revision',
              'Memorize everything',
              "Ignore classmates' help"]},
 {'Question': 'According to the text, how long does Anna spend on revision '
              'each day?',
  'correct_answer': '30 minutes',
  'options': ['15 minutes', '30 minutes', '1 hour', '2 hours']},
 {'Question': 'What is one benefit of revising with a classmate according to '
              'Student 2?',
  'correct_answer': 'It helps to discuss mistakes and errors',
  'options': ['It helps to memorize everything',
              'It allows for more social time',
              'It helps to discuss mistakes and errors',
              'It reduces the amount of work']},
 {'Question': 'What should you avoid doing during revision, as per the advice '
           

In [ ]:
def generate_quiz_all_docs(
                        file_dir = 'English/',
                        difficulty = 'Easy',
                        n_questions = 20
                        ):
    generated_q_dir = 'Generated/{}/'.format(difficulty)
    if not os.path.exists(generated_q_dir):
        os.makedirs(generated_q_dir)

    full_quiz, file_paths = [], []
    for file in os.listdir(file_dir):
        if file.endswith('.pdf'):
            print("Processing file: ", file)
            file_path = file_dir + file
            file_path = file_path.replace('\\', '/')    
            quiz = generate_quiz(
                                file_path = file_dir + file,
                                difficulty = difficulty,
                                generated_q_dir = generated_q_dir
                                )
            full_quiz.extend(quiz)
            file_paths.extend([file_path] * len(quiz))

    print(f"{len(full_quiz)} questions generated ...")
    if len(full_quiz) > n_questions:
        rand_idxs = random.sample(range(len(full_quiz)), n_questions)
        full_quiz = [full_quiz[i] for i in rand_idxs]
        file_paths = [file_paths[i] for i in rand_idxs]
    return full_quiz, file_paths

In [9]:
full_quiz, file_paths = generate_quiz_all_docs()

Processing file:  170210_bbc_learners_questions_unit1.pdf
Processing file:  bbc_exam_skills_revising_part1.pdf
Processing file:  bbc_exam_skills_revising_part1_2.pdf
Processing file:  English questions.pdf
Processing file:  englishclub-7-secrets.pdf
Processing file:  IT1080 -English for Academic Purposes.pdf
Processing file:  IT2090 - Professional Skills.pdf
Processing file:  SE1042-English for Academic Purposes.pdf
Processing file:  t-054-all-tenses.pdf
Processing file:  TENSES (1).pdf
95 questions generated ...


In [10]:
full_quiz

[{'Question': 'What happened after Claire had been running for half an hour?',
  'Answer': 'After Claire had been running for half an hour, she suddenly stopped to check her watch.'},
 {'Question': 'Anna revises daily, spending at least <blank> an hour on different areas of language such as <blank>, reading, vocabulary, and speaking.',
  'Blanks': ['half', 'listening']},
 {'Question': 'What are the benefits of Work From Home (WFH) for employees?',
  'Answer': 'The benefits of Work From Home (WFH) for employees include reduced commuting time and expenses, greater flexibility in work location and hours, fewer distractions, and an improved work-life balance. These factors contribute to increased job satisfaction and productivity.'},
 {'Question': 'What is the main goal of the Biophilic Cities Network?',
  'options': ['To increase urbanization',
   'To reduce greenhouse gas emissions',
   'To integrate nature into urban living',
   'To promote industrial growth'],
  'correct_answer': 'C'},

In [11]:
Q_VALIDATION_PROMPT_TEMPLATE = '''
You are an expert in creating educational content, specializing in creating questions / assignments for English language. You have been given a set of questions to review.
You should validate the questions based on the following criteria:

        1. Grammar and spelling errors
        2. Accuracy / Creditability of the generated questions and answers based on the context
        3. Relevance of the questions to the context

Here you have provided the context and the questions with answers

-----------------------------------------
{context}
-----------------------------------------

{question}

Please provide your feeddback for each of the above criterias out of 10. also provide any additional comments but short if needed.
'''

qval_template = ChatPromptTemplate(
                                    message_templates=[
                                                    ChatMessage(
                                                                role=MessageRole.SYSTEM, 
                                                                content=Q_VALIDATION_PROMPT_TEMPLATE
                                                                )
                                                    ]
                                    )

In [12]:
def question_validation(
                        full_quiz,
                        file_paths
                        ):
    comments = []
    for filepath, q in zip(file_paths, full_quiz):
        context = read_pdf(filepath)
        fmt_messages = qval_template.format_messages(
                                                    context=context,
                                                    question=q
                                                    )
        chat_response = llm.chat(fmt_messages)
        raw_output = chat_response.message.content
        comments.append(raw_output)
    return comments

In [13]:
evaluator = question_validation(full_quiz, file_paths)
evaluator

['1. Grammar and spelling errors: 10/10 \n   - The question and answer are grammatically correct and free of spelling errors.\n\n2. Accuracy / Credibility of the generated questions and answers based on the context: 10/10 \n   - The question accurately reflects the context provided, and the answer is correct according to the given exercise.\n\n3. Relevance of the questions to the context: 10/10 \n   - The question is directly relevant to the context of the exercise, focusing on the specific event that occurred after Claire had been running for half an hour.\n\nAdditional Comments: The question and answer are well-constructed and align perfectly with the context. No improvements needed.',
 "1. Grammar and spelling errors: 10/10\n   - The question and answer are free from any grammar or spelling errors.\n\n2. Accuracy / Credibility of the generated questions and answers based on the context: 10/10\n   - The question accurately reflects the information provided in the context. Anna indeed

In [14]:
print(evaluator[1])

1. Grammar and spelling errors: 10/10
   - The question and answer are free from any grammar or spelling errors.

2. Accuracy / Credibility of the generated questions and answers based on the context: 10/10
   - The question accurately reflects the information provided in the context. Anna indeed revises daily, spending at least half an hour on different areas of language, including listening.

3. Relevance of the questions to the context: 10/10
   - The question is directly relevant to the context, focusing on Anna's revision strategy, which is a key part of the provided text.

Additional Comments:
- The question effectively tests the reader's understanding of the specific details mentioned in the context.


In [15]:
Q_FEEDBACK_PROMPT_TEMPLATE = '''
You are an expert in creating educational content, specializing in evaluating questions / assignments for English language. You have been given a set of answers to review.
You should validate the answers based on the following criteria:

        1. Grammar and spelling errors
        2. Accuracy / Creditability of the generated answers
        3. Relevance of the answer to the question

Here you have provided the candidate answer and the questions with correct answers

-----------------------------------------
{questions}                                                                             
-----------------------------------------

Candidate Answer : {answer}

Please provide your feeddback for each of the above criterias out of 10. also provide any additional comments but short if needed.
'''

qfeedback_template = ChatPromptTemplate(
                                    message_templates=[
                                                    ChatMessage(
                                                                role=MessageRole.SYSTEM, 
                                                                content=Q_FEEDBACK_PROMPT_TEMPLATE
                                                                )
                                                    ]
                                    )

In [16]:
def generate_feedback(
                    question,
                    answer
                    ):
    fmt_messages = qfeedback_template.format_messages(
                                                    questions=question,
                                                    answer=answer
                                                    )
    chat_response = llm.chat(fmt_messages)
    raw_output = chat_response.message.content
    return raw_output

In [17]:
question = full_quiz[3]
question

{'Question': 'What is the main goal of the Biophilic Cities Network?',
 'options': ['To increase urbanization',
  'To reduce greenhouse gas emissions',
  'To integrate nature into urban living',
  'To promote industrial growth'],
 'correct_answer': 'C'}

In [19]:
answer = "C"
feedback = generate_feedback(question, answer)
print(feedback)

Feedback for Candidate Answer:

1. Grammar and Spelling Errors: 10/10
   - The candidate's answer is a single letter, "C," which does not contain any grammar or spelling errors.

2. Accuracy / Credibility of the Generated Answer: 10/10
   - The candidate correctly selected option "C," which aligns with the correct answer provided. The main goal of the Biophilic Cities Network is indeed to integrate nature into urban living.

3. Relevance of the Answer to the Question: 10/10
   - The candidate's answer is directly relevant to the question asked. The selection of option "C" accurately addresses the question regarding the main goal of the Biophilic Cities Network.

Additional Comments:
- The candidate provided the correct answer, and there are no issues with grammar, spelling, accuracy, or relevance. Well done!
